In [1]:
import pandas as pd
import numpy as np
import scipy
import os
import sys
import math
import getopt
import collections
import tqdm
import itertools
import argparse
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import random
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from nltk.tokenize import sent_tokenize, word_tokenize
from scipy import stats
import seaborn as sns; sns.set()
import warnings
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.utils import shuffle
warnings.filterwarnings(action = 'ignore') 
import gensim 
from gensim.models import Word2Vec
import sys
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import RocCurveDisplay
import xgboost

def calc_metrics(cf_matrix):
  tn = cf_matrix[0]
  fp = cf_matrix[1]
  fn = cf_matrix[2]
  tp = cf_matrix[3]
  if (((tp+fn)==0)|((tn+fp)==0)|((tp+fp)==0)|((tn+fp)==0)|((tn+fn)==0)):
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    prec = 0
    acc = (tp+tn)/(tp+tn+fn+fp)
    mcc = 0
    f1 = tp/(tp+(0.5*(fp+fn)))
    return tp,fp,fn,tn,sens,spec,prec,acc,mcc,f1
  else: 
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    prec = tp/(tp+fp)
    acc = (tp+tn)/(tp+tn+fn+fp)
    mcc = ((tp*tn)-(fp*fn))/(((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))**0.5)
    f1 = tp/(tp+(0.5*(fp+fn)))
    return tp,fp,fn,tn,sens,spec,prec,acc,mcc,f1

In [127]:
testing= pd.read_csv("testing_filtered.csv")

In [89]:
te= pd.read_csv("x_test_filtered.csv")
tr= pd.read_csv("x_train_filtered.csv")



In [90]:
x_train = tr.drop(columns=['Label'])
y_train = tr.Label
x_test = te.drop(columns=['Label'])
y_test = te.Label

x_train.reset_index(inplace=True, drop=True)
y_train = y_train.reset_index(drop=True)

x_test.reset_index(inplace=True, drop=True)
y_test = y_test.reset_index(drop=True)



In [92]:
classifier_list = [ExtraTreesClassifier(random_state=1, n_jobs= -1)]

In [93]:
final_metrics = []
for i in classifier_list:
#data_rand
    print(i)
    from  sklearn.model_selection import StratifiedKFold, KFold
    import numpy as np
    from sklearn import svm
    from sklearn.metrics import roc_auc_score
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    #kf.get_n_splits(X)
    cc = []
    dd = []
    ee = []
    clf = i
    auc_scores=[]
    for train, test in skf.split(x_train, y_train):
        train_x = x_train.iloc[train,:]
        test_x = x_train.iloc[test]
        train_y = y_train[train]
        test_y = y_train[test]
        clf.fit(train_x, train_y)

        predict_y = clf.predict_proba(test_x)[:,1]
        cc.append(clf.predict(test_x))
        ee.append(clf.predict_proba(test_x)[:,1])
        dd.append(test_y)
        auc_scores.append(roc_auc_score(test_y, predict_y))
    np.array(auc_scores).mean()
    #Training_matrics
    metrics = []
    for i in range(5):
        metrics.append(calc_metrics(confusion_matrix(dd[i], cc[i]).ravel()))
    train_matrics=pd.DataFrame(metrics, columns=['tp','fp','fn','tn','sens','spec','prec','acc','mcc','f1'])
    asdf = list(train_matrics.mean())
    asdf.append(np.array(auc_scores).mean())
    final_metrics.append(asdf)
    
    #testing metrics
    predict_y = clf.predict_proba(x_test)[:,1]
    predict_label_y = clf.predict(x_test)
    test_auc = roc_auc_score(y_test, predict_y)
    conf_mat_test = confusion_matrix(y_test,predict_label_y).ravel()
    
    test_metrics = list(calc_metrics(conf_mat_test))
    test_metrics.append(test_auc)
    final_metrics.append(test_metrics)

    
pd.DataFrame(final_metrics, columns=['TP','FP','FN','TN','SENS','SPEC','PREC','ACC','MCC','F1','AUC'])

ExtraTreesClassifier(n_jobs=-1, random_state=1)


,TP,FP,FN,TN,SENS,SPEC,PREC,ACC,MCC,F1,AUC
0,547.4,128.0,112.4,555.0,0.829649,0.812592,0.810480,0.820974,0.642272,0.819884,0.905864
1,698.0,163.0,127.0,691.0,0.846061,0.809133,0.810685,0.827278,0.655311,0.827995,0.916144


In [94]:
#Standardization of the columns on the basis of mean and standard deviation ((x-u)/s)
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
scaler = StandardScaler()
#scaler = MinMaxScaler()
#PCA(reduce dimention of feture)
#Imputer(handel nan values)

#for train use fit.transform function
#x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train))
x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train))
#x1.columns=x.columns
#x1
#for test data just .transform function
#x_test_scaled=pd.DataFrame(scaler.transform(x_test)
x_test_scaled = pd.DataFrame(scaler.transform(x_test))


In [95]:
final_metrics = []
for i in classifier_list:
#data_rand
    print(i)
    from  sklearn.model_selection import StratifiedKFold, KFold
    import numpy as np
    from sklearn import svm
    from sklearn.metrics import roc_auc_score
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    #kf.get_n_splits(X)
    cc = []
    dd = []
    ee = []
    clf = i
    auc_scores=[]
    for train, test in skf.split(x_train_scaled, y_train):
        train_x = x_train_scaled.iloc[train,:]
        test_x = x_train_scaled.iloc[test]
        train_y = y_train[train]
        test_y = y_train[test]
        clf.fit(train_x, train_y)

        predict_y = clf.predict_proba(test_x)[:,1]
        cc.append(clf.predict(test_x))
        ee.append(clf.predict_proba(test_x)[:,1])
        dd.append(test_y)
        auc_scores.append(roc_auc_score(test_y, predict_y))
    np.array(auc_scores).mean()
    #Training_matrics
    metrics = []
    for i in range(5):
        metrics.append(calc_metrics(confusion_matrix(dd[i], cc[i]).ravel()))
    train_matrics=pd.DataFrame(metrics, columns=['tp','fp','fn','tn','sens','spec','prec','acc','mcc','f1'])
    asdf = list(train_matrics.mean())
    asdf.append(np.array(auc_scores).mean())
    final_metrics.append(asdf)
    
    #testing metrics
    predict_y = clf.predict_proba(x_test_scaled)[:,1]
    predict_label_y = clf.predict(x_test_scaled)
    test_auc = roc_auc_score(y_test, predict_y)
    conf_mat_test = confusion_matrix(y_test,predict_label_y).ravel()
    
    test_metrics = list(calc_metrics(conf_mat_test))
    test_metrics.append(test_auc)
    final_metrics.append(test_metrics)

    
pd.DataFrame(final_metrics, columns=['TP','FP','FN','TN','SENS','SPEC','PREC','ACC','MCC','F1','AUC'])

ExtraTreesClassifier(n_jobs=-1, random_state=1)


,TP,FP,FN,TN,SENS,SPEC,PREC,ACC,MCC,F1,AUC
0,547.4,128.0,112.4,555.0,0.829649,0.812592,0.810480,0.820974,0.642272,0.819884,0.905864
1,698.0,163.0,127.0,691.0,0.846061,0.809133,0.810685,0.827278,0.655311,0.827995,0.916144


In [96]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


# Define the hyperparameters to search
param_grid = {
    'n_estimators': [7000],

    'random_state': [1]
}

# Create a Gradient Boosting Classifier
clf = ExtraTreesClassifier(n_jobs=-1, random_state=1)


# Create a GridSearchCV object and fit it to the training data
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters
print("Best hyperparameters: ", grid_search.best_params_)

# Evaluate the best model on the test data
best_clf = grid_search.best_estimator_
y_pred_proba = best_clf.predict_proba(x_test)[:, 1]  # predicted probabilities for the positive class
auroc = roc_auc_score(y_test, y_pred_proba)
print("AUROC on test data: ", auroc)


Best hyperparameters:  {'n_estimators': 7000, 'random_state': 1}
AUROC on test data:  0.939146263572493


In [97]:
y_pred_proba

array([0.32514286, 0.50871429, 0.        , ..., 0.688     , 1.        ,
       0.836     ])

In [122]:
aa=(y_pred_proba>0.53).astype(int)

In [123]:
aa

array([0, 0, 0, ..., 1, 1, 1])

In [124]:
from sklearn.metrics import accuracy_score, matthews_corrcoef

accuracy = accuracy_score(y_test, aa)
accuracy

0.8558665872543181

In [128]:
testing_pred_proba = best_clf.predict_proba(testing)[:, 1] 

In [129]:
ab=(testing_pred_proba>0.53).astype(int)

In [133]:
import pandas as pd
df = pd.DataFrame(ab)
df.columns = ['Label']
ID = np.arange(1598) + 10001
ID = pd.DataFrame(ID)
ID.columns = ['ID']
final_data = pd.concat([ID, df], axis=1)
final_data.to_csv('run_14.csv', index=False)